# **Sprint#2. 고객이탈예측 모델링**
- 리워드유효여부에 따라 별도 모델링
- 비시계열/시계열 별도 데이터 셋 구성 

## 데이터셋 구성
- 앱로그
1. 상위 depth별 월평균 체류시간
2. 상위 depth별 세션별 페이지수 (16개 depth)
3. 종료율 

3-1) 고이용군은 높으나 장기미접속군은 낮은 페이지별 종료율

3-2) 장기미접속군은 높으나 고이용군은 낮은 페이지별 종료율

4. 상위 depth별 UV

- 주간미션/리워드
1. 월별 주간미션 달성률
2. 월별 리워드 획득횟수

- 멤버
1. 앱푸쉬 여부(Y/N)
2. 월별 바이탈리티 측정횟수
3. 성별
4. 연령대
5. 현재 등급(4개), 1~4 부여
6. 누적월별 주간미션달성률

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
data_dir = 's3://aiavitality/data/'
fname = 'partyid_monthly_class_210421.csv'
data_path = os.path.join(data_dir, fname)

In [3]:
seg = pd.read_csv(data_path, index_col=0)
seg # 1,524,527

,PartyId,month,class,리워드만료일,리워드유효여부
0,861497,2020-01,0,2019-01-11,N
1,861500,2020-01,0,2019-01-12,N
2,863497,2020-01,0,2019-01-11,N
3,870485,2020-01,0,2019-01-12,N
4,870486,2020-01,0,2019-01-13,N
...,...,...,...,...,...
1524522,1200362289,2021-02,0,2021-05-23,Y
1524523,1200362290,2021-02,0,2021-05-23,Y
1524524,1200362291,2021-02,0,2021-05-23,Y
1524525,1200362301,2021-02,0,2021-05-23,Y


In [4]:
def load_ms(start,end, data_dir):
    # Load mission data
    monthly_df = []
    filename = pd.period_range(start=start, end= end, freq='M').strftime('%Y%m')
    for fn in filename:
        monthly_df.append(pd.read_csv(data_dir + 'mission/goal_misn_' + fn + '.csv',
                                      usecols= ['party_id', 'goal_id', 'objective_state',
                                      'goal_end_dt', 'objective_status_chg_dt',
                                      'vchr_dc', 'vchr_sta_dt'], 
                                      dtype = {"party_id" : "str"}))
    df = pd.concat(monthly_df)
    return df

In [5]:
def basic_preprocess_ms(df, end_month):
    # 칼럼명 변경
    df = df.rename(columns = { 'party_id':'PartyId','objective_state': '주간미션상태',
                          'goal_end_dt': '미션종료기한일','objective_status_chg_dt' : '주간미션연동일',
                          'vchr_dc' : '리워드설명', 'vchr_sta_dt': '리워드시작일'}, inplace = False)
    df = df[df.goal_id != '#']
    df = df.drop_duplicates()
    
    # 일자 변환
    df['미션종료기한일']= df['미션종료기한일'].astype('str')
    df['주간미션연동일']= df['주간미션연동일'].astype('str')
    
    df['미션종료기한일'] = pd.to_datetime(df['미션종료기한일'],format='%Y%m%d')
    df['주간미션연동일'] = pd.to_datetime(df['주간미션연동일'],format='%Y%m%d')
    
    # 주차 정보 추가
    df['Week'] = pd.factorize(df['미션종료기한일'].dt.weekofyear)[0] + 1

    # month 추가
    df['month'] = df['미션종료기한일'].dt.strftime('%Y-%m')
    df = df[df['month'] < end_month]
    return df

In [8]:
def merge_ms_reward(reward,ms):
    reward['PartyId'] = reward['PartyId'].astype('str')
    ms['PartyId'] = ms['PartyId'].astype('str')
    ms_summary = pd.merge(reward, ms,how='inner',on=["PartyId",'month'])
    
    ms_summary['리워드획득여부']=ms_summary['리워드획득여부'].astype('str')
    ms_summary['리워드획득여부'] = np.where(ms_summary['리워드획득여부'] == "1.0","1", "0")
    ms_summary['월별리워드획득횟수'] = ms_summary['월별리워드획득횟수'].replace(np.nan, 0)
    
    ms_summary = ms_summary[['PartyId','month','월별리워드획득횟수','주간미션달성횟수','월별미션달성률']]
    ms_summary = ms_summary.drop_duplicates()
    return ms_summary

## 멤버데이터
1. 앱푸쉬 여부(Y/N): 월평균으로 변환 
2. 월별 바이탈리티 측정횟수 , vtlt_age_eff_dt
3. 성별
4. 연령대
5. 현재 등급(4개), 1~4 부여 Bronze < Silver < Gold < Platinum , cur_mbrsh_rwrd_st_cd, 월평균으로 변환
6. 누적월별 주간미션달성률, achv_rat

In [4]:
def load_mbr(start,end,data_dir):
    # Load mission data
    monthly_df = []
    filename = pd.period_range(start=start, end= end, freq='M').strftime('%Y%m')
    for fn in filename:
        monthly_df.append(pd.read_csv(data_dir + 'member/mbr_' + fn + '.csv',
                                      usecols= ['dt','party_id', 'gender_cd', 'age', 
                                                'vtlt_age_eff_dt','cur_mbrsh_rwrd_st_cd','push_alarm_yn',
                                                'cur_mbrsh_pd_goal_achv_cnt','cur_mbrsh_pd_goal_alct_cnt'])) 
    df = pd.concat(monthly_df)
    return df

In [21]:
start = 202101
end = 202102

data_dir = 's3://aiavitality/data/'
mbr = load_mbr(start,end, data_dir)
mbr

,party_id,gender_cd,age,vtlt_age_eff_dt,cur_mbrsh_rwrd_st_cd,cur_mbrsh_pd_goal_alct_cnt,cur_mbrsh_pd_goal_achv_cnt,push_alarm_yn,dt
0,1048481,FEMALE,39,20190927,Bronze,127,25,N,20210101
1,1359484,FEMALE,34,20180811,Bronze,126,20,Y,20210101
2,2687982,FEMALE,87,20201108,Bronze,124,51,Y,20210101
3,3432997,FEMALE,43,20191018,Bronze,124,22,N,20210101
4,3492989,MALE,49,20191206,Silver,124,28,N,20210101
...,...,...,...,...,...,...,...,...,...
2234069,1200358128,FEMALE,43,20210223,Bronze,1,0,Y,20210228
2234070,1200358196,FEMALE,50,20210224,Bronze,1,0,Y,20210228
2234071,1200361377,MALE,33,20210227,Bronze,1,0,Y,20210228
2234072,1200361438,FEMALE,25,20210227,Bronze,1,0,Y,20210228


In [22]:
def basic_preprocess_mbr(mbrdf, end_month):
    # 칼럼명 변경
    mbrdf = mbrdf.rename(columns = { 'party_id':'PartyId','cur_mbrsh_rwrd_st_cd': '멤버십등급',
                              'vtlt_age_eff_dt': '바이탈리티나이측정일자','push_alarm_yn' : '앱푸쉬여부',
                              'cur_mbrsh_pd_goal_achv_cnt': '현재멤버십기간활동목표달성수', 'cur_mbrsh_pd_goal_alct_cnt': '현재멤버십기간활동목표할당수'}, inplace = False)
    mbrdf = mbrdf[mbrdf['멤버십등급']!="#"]
    # 일자 변환
    mbrdf['dt']= mbrdf['dt'].astype('str')
    mbrdf['PartyId']= mbrdf['PartyId'].astype('int64')
    
    mbrdf['dt'] = pd.to_datetime(mbrdf['dt'],format='%Y%m%d')

    # month 추가
    mbrdf['month'] = mbrdf['dt'].dt.strftime('%Y-%m')
    mbrdf = mbrdf[mbrdf['month'] < end_month]
    
    # age_group 추가
    mbrdf['연령대'] = mbrdf['age'] // 10
    
    mbrdf = mbrdf.sort_values(['PartyId', 'dt'])   
    
    # party_id -> int형으로 변환
    mbrdf['PartyId'] = mbrdf['PartyId'].astype('int32')
    
    def count_vtlt_age_eff_dt(x):
        count_vtlt_age = np.zeros(len(x['바이탈리티나이측정일자']), dtype=np.float32)
        vtlt_effs = np.unique(x['바이탈리티나이측정일자'])
        for eff in vtlt_effs:
            if eff == 99991231:
                continue
            else:
                ind = np.where(x['바이탈리티나이측정일자'] == eff)[0][0]
                count_vtlt_age[ind:] += 1
        return pd.Series(count_vtlt_age, name='바이탈리티나이측정횟수')
    
    # 바이탈리티 나이 측정 횟수 관련 전처리
    newcol = mbrdf.groupby(['PartyId']).apply(count_vtlt_age_eff_dt)
    mbrdf['바이탈리티나이측정횟수'] = newcol.values
    mbrdf['월바이탈리티나이측정횟수'] = mbrdf.groupby(['PartyId','month'])['바이탈리티나이측정횟수'].transform('max')
    
    # 주간미션달성률 관련 전처리
    mbrdf['누적주간미션달성률'] = round(mbrdf['현재멤버십기간활동목표달성수'] / mbrdf['현재멤버십기간활동목표할당수'],4)
    mbrdf['누적주간미션달성률'] = mbrdf['누적주간미션달성률'].replace(np.nan, 0)
    mbrdf['월평균누적주간미션달성률'] = round(mbrdf.groupby(['PartyId','month'])['누적주간미션달성률'].transform('mean'),4)
    
    # 멤버십 등급 -> 1,2,3,4로 변경
    mbrsh_dic = {'Bronze': 1, 'Silver': 2, 'Gold': 3, 'Platinum': 4}
    f = lambda x : mbrsh_dic[x]
    newmbrsh = mbrdf['멤버십등급'].transform(f)
    mbrdf['멤버십등급'] = newmbrsh
    mbrdf['월평균멤버십등급'] = round(mbrdf.groupby(['PartyId','month'])['멤버십등급'].transform('mean'), 4)
    
    # 앱푸쉬여부 ->1,0으로 변경
    app_dic = {'Y': 1, 'N': 0}
    f = lambda x : app_dic[x]
    newapp = mbrdf['앱푸쉬여부'].transform(f)
    mbrdf['앱푸쉬여부'] = newapp
    mbrdf['월평균앱푸쉬여부'] = round(mbrdf.groupby(['PartyId','month'])['앱푸쉬여부'].transform('mean'), 4)
    
    # 성별 ->1,0으로 변경
    gender_dic = {'MALE': 1, 'FEMALE': 0}
    f = lambda x : gender_dic[x]
    newgen = mbrdf['gender_cd'].transform(f)
    mbrdf['gender_cd'] = newgen
    
    
    # 필요없는 칼럼 drop
    mbrdf = mbrdf.drop(columns=['dt','age','바이탈리티나이측정일자', '바이탈리티나이측정횟수' ,
                                '현재멤버십기간활동목표달성수','현재멤버십기간활동목표할당수','누적주간미션달성률',
                                '멤버십등급','앱푸쉬여부'])
    mbrdf = mbrdf.drop_duplicates(['PartyId','month'],keep ='last')
    return mbrdf

In [23]:
end_month = '2021-03'
mbrdf = basic_preprocess_mbr(mbr,end_month)
mbrdf

,PartyId,gender_cd,month,연령대,월바이탈리티나이측정횟수,월평균누적주간미션달성률,월평균멤버십등급,월평균앱푸쉬여부
3215295,861497,1,2021-01,3,1.0,0.3227,1.0,1.0
2209542,861497,1,2021-02,3,1.0,0.3133,1.0,1.0
3251315,863496,1,2021-01,3,1.0,0.1771,1.0,1.0
2228241,863496,1,2021-02,3,1.0,0.1715,1.0,1.0
3220023,863497,1,2021-01,5,1.0,0.1540,1.0,1.0
...,...,...,...,...,...,...,...,...
2187612,1200362289,1,2021-02,7,1.0,0.0000,1.0,1.0
2168767,1200362290,1,2021-02,2,1.0,0.0000,1.0,1.0
2228240,1200362291,0,2021-02,4,1.0,0.0000,1.0,1.0
2184252,1200362301,1,2021-02,3,1.0,0.0000,1.0,1.0


In [24]:
mbrdf['PartyId'].value_counts()

201328639     2
21838485      2
201056846     2
200876127     2
200739411     2
             ..
1200315252    1
1200319346    1
1200158540    1
1200327534    1
257171897     1
Name: PartyId, Length: 130492, dtype: int64

In [25]:
mbrdf[mbrdf['PartyId']== 1200157468]

,PartyId,gender_cd,month,연령대,월바이탈리티나이측정횟수,월평균누적주간미션달성률,월평균멤버십등급,월평균앱푸쉬여부
3257634,1200157468,0,2021-01,3,1.0,0.0,1.0,1.0
2109242,1200157468,0,2021-02,4,1.0,0.0,1.0,1.0


In [26]:
month = str(start) + '_' +str(end)
month
# mbrdf.to_csv('result/mbrdf_' + month + '.csv')

'202101_202102'

In [11]:
def merge_seg(df,seg):    
    seg['PartyId'] = seg['PartyId'].astype('str')
    df['PartyId'] = df['PartyId'].astype(float).astype(int)
    df['PartyId'] = df['PartyId'].astype('str')
    df_seg = pd.merge(df, seg, how='inner',on=["PartyId", "month"])
    return df_seg

In [27]:
mbr_seg = merge_seg(mbrdf,seg)
mbr_seg.to_csv('result/mbrseg_' + month + '.csv')

In [28]:
mbr_seg

,PartyId,gender_cd,month,연령대,월바이탈리티나이측정횟수,월평균누적주간미션달성률,월평균멤버십등급,월평균앱푸쉬여부,class,리워드만료일,리워드유효여부
0,861497,1,2021-01,3,1.0,0.3227,1.0,1.0,0,2019-01-11,N
1,861497,1,2021-02,3,1.0,0.3133,1.0,1.0,0,2019-01-11,N
2,863496,1,2021-01,3,1.0,0.1771,1.0,1.0,0,2019-01-11,N
3,863496,1,2021-02,3,1.0,0.1715,1.0,1.0,0,2019-01-11,N
4,863497,1,2021-01,5,1.0,0.1540,1.0,1.0,0,2019-01-11,N
...,...,...,...,...,...,...,...,...,...,...,...
115466,1200362289,1,2021-02,7,1.0,0.0000,1.0,1.0,0,2021-05-23,Y
115467,1200362290,1,2021-02,2,1.0,0.0000,1.0,1.0,0,2021-05-23,Y
115468,1200362291,0,2021-02,4,1.0,0.0000,1.0,1.0,0,2021-05-23,Y
115469,1200362301,1,2021-02,3,1.0,0.0000,1.0,1.0,0,2021-05-23,Y


In [67]:
def concat_mbr(start,end,data_dir):
    monthly_df = []
    filename = ['202001','202004','202007','202010', '202101']
    for fn in filename:
        monthly_df.append(pd.read_csv(data_dir + 'mbrseg_' + fn + '.csv', index_col=0))
        df = pd.concat(monthly_df)
        df = df.drop_duplicates()
    return df

In [68]:
start = 202001
end = 202101
data_dir = 'result/'
mbr_all = concat_mbr(start,end,data_dir)
mbr_all

,PartyId,gender_cd,month,연령대,월바이탈리티나이측정횟수,월평균누적주간미션달성률,월평균멤버십등급,월평균앱푸쉬여부,class,리워드만료일,리워드유효여부
0,861497,1,2020-01,3,1.0,0.3849,3.0,1.0,0,2019-01-11,N
1,861497,1,2020-02,3,1.0,0.4008,3.0,1.0,0,2019-01-11,N
2,861497,1,2020-03,3,1.0,0.3828,3.0,1.0,0,2019-01-11,N
3,861500,1,2020-01,5,1.0,0.4715,4.0,1.0,0,2019-01-12,N
4,861500,1,2020-02,5,1.0,0.4588,4.0,1.0,0,2019-01-12,N
...,...,...,...,...,...,...,...,...,...,...,...
115466,1200362289,1,2021-02,7,1.0,0.0000,1.0,1.0,0,2021-05-23,Y
115467,1200362290,1,2021-02,2,1.0,0.0000,1.0,1.0,0,2021-05-23,Y
115468,1200362291,0,2021-02,4,1.0,0.0000,1.0,1.0,0,2021-05-23,Y
115469,1200362301,1,2021-02,3,1.0,0.0000,1.0,1.0,0,2021-05-23,Y


In [69]:
mbr_all = mbr_all.sort_values(by=['PartyId', 'month'])
mbr_all

,PartyId,gender_cd,month,연령대,월바이탈리티나이측정횟수,월평균누적주간미션달성률,월평균멤버십등급,월평균앱푸쉬여부,class,리워드만료일,리워드유효여부
0,861497,1,2020-01,3,1.0,0.3849,3.0,1.0,0,2019-01-11,N
1,861497,1,2020-02,3,1.0,0.4008,3.0,1.0,0,2019-01-11,N
2,861497,1,2020-03,3,1.0,0.3828,3.0,1.0,0,2019-01-11,N
0,861497,1,2020-04,3,1.0,0.3751,3.0,1.0,0,2019-01-11,N
1,861497,1,2020-05,3,1.0,0.3690,3.0,1.0,0,2019-01-11,N
...,...,...,...,...,...,...,...,...,...,...,...
115466,1200362289,1,2021-02,7,1.0,0.0000,1.0,1.0,0,2021-05-23,Y
115467,1200362290,1,2021-02,2,1.0,0.0000,1.0,1.0,0,2021-05-23,Y
115468,1200362291,0,2021-02,4,1.0,0.0000,1.0,1.0,0,2021-05-23,Y
115469,1200362301,1,2021-02,3,1.0,0.0000,1.0,1.0,0,2021-05-23,Y


In [70]:
mbr_all['PartyId'].value_counts()

201328639     14
200241361     14
102272595     14
200339295     14
200275529     14
              ..
200221757      1
200062031      1
1200360536     1
200031326      1
1200184104     1
Name: PartyId, Length: 527824, dtype: int64

In [71]:
ms_seg = pd.read_csv(data_dir + 'ms_seg_202001_202103.csv', index_col=0)
ms_seg

,PartyId,month,월별리워드획득횟수,주간미션달성횟수,월별미션달성률,class,리워드만료일,리워드유효여부
0,75379586,2020-01,4,3,1.00,0,2019-02-18,N
1,200170086,2020-01,3,3,1.00,0,2019-05-07,N
2,200368551,2020-01,4,3,1.00,0,2019-06-11,N
3,200374628,2020-01,3,3,1.00,0,2019-06-13,N
4,200379789,2020-01,3,3,1.00,0,2019-06-14,N
...,...,...,...,...,...,...,...,...
324486,95769084,2021-02,1,1,0.25,0,2019-02-20,N
324487,201302639,2021-02,1,1,0.25,0,2020-02-07,N
324488,200503651,2021-02,1,1,0.25,0,2019-07-15,N
324489,200378467,2021-02,1,1,0.25,0,2019-06-14,N


In [76]:
ms_seg = ms_seg[['PartyId','month','월별리워드획득횟수','월별미션달성률']]
ms_seg['PartyId'] = ms_seg['PartyId'].astype('str')
mbr_all['PartyId'] = mbr_all['PartyId'].astype('str')
df_all = pd.merge(mbr_all, ms_seg, how='left',on=["PartyId", "month"])

In [77]:
df_all

,PartyId,gender_cd,month,연령대,월바이탈리티나이측정횟수,월평균누적주간미션달성률,월평균멤버십등급,월평균앱푸쉬여부,class,리워드만료일,리워드유효여부,월별리워드획득횟수,월별미션달성률
0,861497,1,2020-01,3,1.0,0.3849,3.0,1.0,0,2019-01-11,N,NaN,NaN
1,861497,1,2020-02,3,1.0,0.4008,3.0,1.0,0,2019-01-11,N,NaN,NaN
2,861497,1,2020-03,3,1.0,0.3828,3.0,1.0,0,2019-01-11,N,NaN,NaN
3,861497,1,2020-04,3,1.0,0.3751,3.0,1.0,0,2019-01-11,N,NaN,NaN
4,861497,1,2020-05,3,1.0,0.3690,3.0,1.0,0,2019-01-11,N,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523876,1200362289,1,2021-02,7,1.0,0.0000,1.0,1.0,0,2021-05-23,Y,NaN,NaN
1523877,1200362290,1,2021-02,2,1.0,0.0000,1.0,1.0,0,2021-05-23,Y,NaN,NaN
1523878,1200362291,0,2021-02,4,1.0,0.0000,1.0,1.0,0,2021-05-23,Y,NaN,NaN
1523879,1200362301,1,2021-02,3,1.0,0.0000,1.0,1.0,0,2021-05-23,Y,NaN,NaN


In [78]:
df_all['월별리워드획득횟수'] = df_all['월별리워드획득횟수'].replace(np.nan, 0)
df_all['월별미션달성률'] = df_all['월별미션달성률'].replace(np.nan, 0)
df_all

,PartyId,gender_cd,month,연령대,월바이탈리티나이측정횟수,월평균누적주간미션달성률,월평균멤버십등급,월평균앱푸쉬여부,class,리워드만료일,리워드유효여부,월별리워드획득횟수,월별미션달성률
0,861497,1,2020-01,3,1.0,0.3849,3.0,1.0,0,2019-01-11,N,0.0,0.0
1,861497,1,2020-02,3,1.0,0.4008,3.0,1.0,0,2019-01-11,N,0.0,0.0
2,861497,1,2020-03,3,1.0,0.3828,3.0,1.0,0,2019-01-11,N,0.0,0.0
3,861497,1,2020-04,3,1.0,0.3751,3.0,1.0,0,2019-01-11,N,0.0,0.0
4,861497,1,2020-05,3,1.0,0.3690,3.0,1.0,0,2019-01-11,N,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523876,1200362289,1,2021-02,7,1.0,0.0000,1.0,1.0,0,2021-05-23,Y,0.0,0.0
1523877,1200362290,1,2021-02,2,1.0,0.0000,1.0,1.0,0,2021-05-23,Y,0.0,0.0
1523878,1200362291,0,2021-02,4,1.0,0.0000,1.0,1.0,0,2021-05-23,Y,0.0,0.0
1523879,1200362301,1,2021-02,3,1.0,0.0000,1.0,1.0,0,2021-05-23,Y,0.0,0.0


In [79]:
df_all.to_csv('result/ms_mbr_202001_202102.csv')

In [82]:
df_all.describe()

,gender_cd,연령대,월바이탈리티나이측정횟수,월평균누적주간미션달성률,월평균멤버십등급,월평균앱푸쉬여부,class,월별리워드획득횟수,월별미션달성률
count,1.523881e+06,1.523881e+06,1.523881e+06,1.523881e+06,1.523881e+06,1.523881e+06,1.523881e+06,1.523881e+06,1.523881e+06
mean,4.783333e-01,3.838467e+00,1.101931e+00,2.930349e-01,1.327504e+00,7.696935e-01,4.594742e-01,5.492056e-01,1.627445e-01
std,4.995305e-01,1.269764e+00,4.774352e-01,3.051797e-01,5.536821e-01,4.028422e-01,4.983551e-01,1.214176e+00,3.378044e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,7.667000e-01,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,4.000000e+00,1.000000e+00,1.924000e-01,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.000000e+00,5.000000e+00,1.000000e+00,4.965000e-01,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,9.000000e+00,8.400000e+01,9.912000e-01,4.000000e+00,1.000000e+00,1.000000e+00,6.000000e+00,1.000000e+00


In [80]:
df_all['월별미션달성률'].value_counts()

0.0000    1201175
1.0000     160139
0.7500      34952
0.5000      29644
0.2500      21244
0.8000      18461
0.6000      17691
0.4000      15533
0.2000      11099
0.6667       9159
0.3333       4784
Name: 월별미션달성률, dtype: int64

In [81]:
df_all['월별리워드획득횟수'].value_counts()

0.0    1199390
1.0      87362
2.0      83162
4.0      70725
3.0      58019
5.0      25057
6.0        166
Name: 월별리워드획득횟수, dtype: int64

In [ ]:
# 차이 데이터 구하기 (이번달 - 이전월)

In [95]:
data_dir = 's3://aiavitality/data/mission'
fname = 'goal_misn_202011.csv'
data_path = os.path.join(data_dir, fname)
df = pd.read_csv(data_path, usecols= ['dt','party_id', 'goal_id', 'objective_state',
                                      'goal_end_dt', 'objective_status_chg_dt',
                                      'vchr_dc', 'vchr_sta_dt'])
# 컬럼명 변경
df = df.rename(columns = { 'party_id':'PartyId',
                          'objective_state': '주간미션상태','points_achieved': '주간미션포인트획득상태',
                          'goal_end_dt': '미션종료기한일','objective_status_chg_dt' : '주간미션연동일',
                          'vchr_dc' : '리워드설명', 'vchr_sta_dt': '리워드시작일'}, inplace = False)
df

,PartyId,goal_id,주간미션상태,미션종료기한일,주간미션연동일,리워드설명,리워드시작일,dt
0,892495,1255559564,Achieved,20201108,20201109,#,#,20201102
1,2423482,1253718066,Achieved,20201108,20201105,#,#,20201102
2,2916984,1253715006,Not Achieved,20201108,20201029,#,#,20201102
3,9141996,1255441584,Achieved,20201108,20201106,SK Telecom Voucher,20201109,20201102
4,11825490,1255706054,Achieved,20201108,20201108,#,#,20201102
...,...,...,...,...,...,...,...,...
933128,1200265690,1264101570,Achieved,20201206,20201204,SK Telecom Voucher,20201204,20201130
933129,1200271414,1265773663,Not Achieved,20201206,20201127,#,#,20201130
933130,1200279229,1264025570,Achieved,20201206,20201202,SK Telecom Voucher,20201202,20201130
933131,1200279565,1266020118,Achieved,20201206,20201207,#,#,20201130


## 전체 전처리
1. 일자 변환
2. 주차 추가
3. 월 추가 

In [96]:
df = df[df.goal_id != '#']

# 일자 변환
df['미션종료기한일']=df['미션종료기한일'].astype('str')
df['주간미션연동일']=df['주간미션연동일'].astype('str')
df['dt']=df['dt'].astype('str')

df['미션종료기한일'] = pd.to_datetime(df['미션종료기한일'],format='%Y%m%d')
df['주간미션연동일'] = pd.to_datetime(df['주간미션연동일'],format='%Y%m%d')
df['dt'] = pd.to_datetime(df['dt'],format='%Y%m%d')

# 주차 정보 추가
df['Week'] = pd.factorize(df['미션종료기한일'].dt.weekofyear)[0] + 1

# month 추가
df['month'] = df['미션종료기한일'].dt.strftime('%Y-%m')

In [97]:
df.head()

,PartyId,goal_id,주간미션상태,미션종료기한일,주간미션연동일,리워드설명,리워드시작일,dt,Week,month
0,892495,1255559564,Achieved,2020-11-08,2020-11-09,#,#,2020-11-02,1,2020-11
1,2423482,1253718066,Achieved,2020-11-08,2020-11-05,#,#,2020-11-02,1,2020-11
2,2916984,1253715006,Not Achieved,2020-11-08,2020-10-29,#,#,2020-11-02,1,2020-11
3,9141996,1255441584,Achieved,2020-11-08,2020-11-06,SK Telecom Voucher,20201109,2020-11-02,1,2020-11
4,11825490,1255706054,Achieved,2020-11-08,2020-11-08,#,#,2020-11-02,1,2020-11


## 주간미션 분석항목
1. 월별 주간미션 달성률
2. 월별 리워드 획득횟수

### 1.월별 주간미션 달성률

In [98]:
ms = df[['PartyId','goal_id','Week','month','주간미션상태','미션종료기한일','주간미션연동일']]
ms = ms.drop_duplicates()

ms['주간미션달성여부'] = np.where(ms['주간미션상태'] == "Achieved","1", "0")
ms['주간미션달성여부'] = ms['주간미션달성여부'].astype('int')
ms['주간미션달성횟수'] = ms.groupby(['PartyId','month'])['주간미션달성여부'].transform('sum')
ms['월별최대달성횟수'] = ms.groupby(['month'])['주간미션달성횟수'].transform('max')

ms['월별미션달성률'] = round(ms['주간미션달성횟수']/ms['월별최대달성횟수']*100,1)
ms

,PartyId,goal_id,Week,month,주간미션상태,미션종료기한일,주간미션연동일,주간미션달성여부,주간미션달성횟수,월별최대달성횟수,월별미션달성률
0,892495,1255559564,1,2020-11,Achieved,2020-11-08,2020-11-09,1,2,4,50.0
1,2423482,1253718066,1,2020-11,Achieved,2020-11-08,2020-11-05,1,3,4,75.0
2,2916984,1253715006,1,2020-11,Not Achieved,2020-11-08,2020-10-29,0,0,4,0.0
3,9141996,1255441584,1,2020-11,Achieved,2020-11-08,2020-11-06,1,4,4,100.0
4,11825490,1255706054,1,2020-11,Achieved,2020-11-08,2020-11-08,1,4,4,100.0
...,...,...,...,...,...,...,...,...,...,...,...
933126,1200265690,1264101570,5,2020-12,Achieved,2020-12-06,2020-12-04,1,1,1,100.0
933129,1200271414,1265773663,5,2020-12,Not Achieved,2020-12-06,2020-11-27,0,0,1,0.0
933130,1200279229,1264025570,5,2020-12,Achieved,2020-12-06,2020-12-02,1,1,1,100.0
933131,1200279565,1266020118,5,2020-12,Achieved,2020-12-06,2020-12-07,1,1,1,100.0


In [99]:
ms['PartyId']=ms['PartyId'].astype('str')
ms[ms['PartyId']=='892495']

,PartyId,goal_id,Week,month,주간미션상태,미션종료기한일,주간미션연동일,주간미션달성여부,주간미션달성횟수,월별최대달성횟수,월별미션달성률
0,892495,1255559564,1,2020-11,Achieved,2020-11-08,2020-11-09,1,2,4,50.0
280112,892495,1258464120,2,2020-11,Achieved,2020-11-15,2020-11-14,1,2,4,50.0
544913,892495,1260852568,3,2020-11,Not Achieved,2020-11-22,2020-11-14,0,2,4,50.0


### 2. 월별 리워드 획득횟수

In [102]:
# 활동별로 파일 쪼개기
Reward = df[['Week','PartyId','리워드설명','리워드시작일']]
Reward =Reward[Reward['리워드설명']!='#']
Reward['리워드시작일']= Reward['리워드시작일'].astype('str')
Reward['리워드시작일'] = pd.to_datetime(Reward['리워드시작일'],format='%Y%m%d')
# month 추가
Reward['month'] = Reward['리워드시작일'].dt.strftime('%Y-%m')
Reward = Reward.drop_duplicates()
Reward 

,Week,PartyId,리워드설명,리워드시작일,month
3,1,9141996,SK Telecom Voucher,2020-11-09,2020-11
7,1,46712094,SK Telecom Voucher,2020-11-06,2020-11
8,1,48541598,SK Telecom Voucher,2020-11-08,2020-11
10,1,49322580,Starbucks Voucher,2020-11-11,2020-11
15,1,100402592,SK Telecom Voucher,2020-11-07,2020-11
...,...,...,...,...,...
933123,5,1200262391,SK Telecom Voucher,2020-12-03,2020-12
933124,5,1200264075,SK Telecom Voucher,2020-12-06,2020-12
933125,5,1200264253,SK Telecom Voucher,2020-12-04,2020-12
933126,5,1200265690,SK Telecom Voucher,2020-12-04,2020-12


In [103]:
Reward['리워드획득여부']='1'
Reward['리워드획득여부'] = Reward['리워드획득여부'].astype('int')
Reward['월별리워드획득횟수'] = Reward.groupby(['PartyId','month'])['리워드획득여부'].transform('sum')
Reward

,Week,PartyId,리워드설명,리워드시작일,month,리워드획득여부,월별리워드획득횟수
3,1,9141996,SK Telecom Voucher,2020-11-09,2020-11,1,4
7,1,46712094,SK Telecom Voucher,2020-11-06,2020-11,1,4
8,1,48541598,SK Telecom Voucher,2020-11-08,2020-11,1,4
10,1,49322580,Starbucks Voucher,2020-11-11,2020-11,1,2
15,1,100402592,SK Telecom Voucher,2020-11-07,2020-11,1,4
...,...,...,...,...,...,...,...
933123,5,1200262391,SK Telecom Voucher,2020-12-03,2020-12,1,1
933124,5,1200264075,SK Telecom Voucher,2020-12-06,2020-12,1,1
933125,5,1200264253,SK Telecom Voucher,2020-12-04,2020-12,1,1
933126,5,1200265690,SK Telecom Voucher,2020-12-04,2020-12,1,1


In [104]:
Reward['PartyId'] = Reward['PartyId'].astype('str')
ms['PartyId'] = ms['PartyId'].astype('str')
ms_summary = pd.merge(Reward, ms,how='outer',on=["PartyId", "Week",'month'])
ms_summary

,Week,PartyId,리워드설명,리워드시작일,month,리워드획득여부,월별리워드획득횟수,goal_id,주간미션상태,미션종료기한일,주간미션연동일,주간미션달성여부,주간미션달성횟수,월별최대달성횟수,월별미션달성률
0,1,9141996,SK Telecom Voucher,2020-11-09,2020-11,1.0,4.0,1255441584,Achieved,2020-11-08,2020-11-06,1.0,4.0,4.0,100.0
1,1,46712094,SK Telecom Voucher,2020-11-06,2020-11,1.0,4.0,1255368147,Achieved,2020-11-08,2020-11-06,1.0,4.0,4.0,100.0
2,1,48541598,SK Telecom Voucher,2020-11-08,2020-11,1.0,4.0,1255641614,Achieved,2020-11-08,2020-11-07,1.0,4.0,4.0,100.0
3,1,49322580,Starbucks Voucher,2020-11-11,2020-11,1.0,2.0,1255655086,Achieved,2020-11-08,2020-11-09,1.0,3.0,4.0,75.0
4,1,100402592,SK Telecom Voucher,2020-11-07,2020-11,1.0,4.0,1255727119,Achieved,2020-11-08,2020-11-07,1.0,4.0,4.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207490,5,1200251692,NaN,NaT,2020-12,NaN,NaN,1266108557,Not Achieved,2020-12-06,2020-11-29,0.0,0.0,1.0,0.0
207491,5,1200259349,NaN,NaT,2020-12,NaN,NaN,1264459320,Not Achieved,2020-12-06,2020-11-26,0.0,0.0,1.0,0.0
207492,5,1200271414,NaN,NaT,2020-12,NaN,NaN,1265773663,Not Achieved,2020-12-06,2020-11-27,0.0,0.0,1.0,0.0
207493,5,1200279565,NaN,NaT,2020-12,NaN,NaN,1266020118,Achieved,2020-12-06,2020-12-07,1.0,1.0,1.0,100.0


In [105]:
ms_summary['리워드획득여부']=ms_summary['리워드획득여부'].astype('str')
ms_summary['리워드획득여부'] = np.where(ms_summary['리워드획득여부'] == "1.0","1", "0")
import numpy as np
ms_summary['월별리워드획득횟수'] = ms_summary['월별리워드획득횟수'].replace(np.nan, 0)
ms_summary

,Week,PartyId,리워드설명,리워드시작일,month,리워드획득여부,월별리워드획득횟수,goal_id,주간미션상태,미션종료기한일,주간미션연동일,주간미션달성여부,주간미션달성횟수,월별최대달성횟수,월별미션달성률
0,1,9141996,SK Telecom Voucher,2020-11-09,2020-11,1,4.0,1255441584,Achieved,2020-11-08,2020-11-06,1.0,4.0,4.0,100.0
1,1,46712094,SK Telecom Voucher,2020-11-06,2020-11,1,4.0,1255368147,Achieved,2020-11-08,2020-11-06,1.0,4.0,4.0,100.0
2,1,48541598,SK Telecom Voucher,2020-11-08,2020-11,1,4.0,1255641614,Achieved,2020-11-08,2020-11-07,1.0,4.0,4.0,100.0
3,1,49322580,Starbucks Voucher,2020-11-11,2020-11,1,2.0,1255655086,Achieved,2020-11-08,2020-11-09,1.0,3.0,4.0,75.0
4,1,100402592,SK Telecom Voucher,2020-11-07,2020-11,1,4.0,1255727119,Achieved,2020-11-08,2020-11-07,1.0,4.0,4.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207490,5,1200251692,NaN,NaT,2020-12,0,0.0,1266108557,Not Achieved,2020-12-06,2020-11-29,0.0,0.0,1.0,0.0
207491,5,1200259349,NaN,NaT,2020-12,0,0.0,1264459320,Not Achieved,2020-12-06,2020-11-26,0.0,0.0,1.0,0.0
207492,5,1200271414,NaN,NaT,2020-12,0,0.0,1265773663,Not Achieved,2020-12-06,2020-11-27,0.0,0.0,1.0,0.0
207493,5,1200279565,NaN,NaT,2020-12,0,0.0,1266020118,Achieved,2020-12-06,2020-12-07,1.0,1.0,1.0,100.0


In [106]:
ms_summary.columns

Index(['Week', 'PartyId', '리워드설명', '리워드시작일', 'month', '리워드획득여부', '월별리워드획득횟수',
       'goal_id', '주간미션상태', '미션종료기한일', '주간미션연동일', '주간미션달성여부', '주간미션달성횟수',
       '월별최대달성횟수', '월별미션달성률'],
      dtype='object')

In [107]:
ms_summary = ms_summary[['PartyId','month','월별리워드획득횟수','주간미션달성횟수','월별미션달성률']]
ms_summary = ms_summary.drop_duplicates()
ms_summary

,PartyId,month,월별리워드획득횟수,주간미션달성횟수,월별미션달성률
0,9141996,2020-11,4.0,4.0,100.0
1,46712094,2020-11,4.0,4.0,100.0
2,48541598,2020-11,4.0,4.0,100.0
3,49322580,2020-11,2.0,3.0,75.0
4,100402592,2020-11,4.0,4.0,100.0
...,...,...,...,...,...
207490,1200251692,2020-12,0.0,0.0,0.0
207491,1200259349,2020-12,0.0,0.0,0.0
207492,1200271414,2020-12,0.0,0.0,0.0
207493,1200279565,2020-12,0.0,1.0,100.0


In [108]:
seg

,PartyId,month,class
0,861497,2020-01,0
1,861500,2020-01,0
2,863497,2020-01,0
3,870485,2020-01,0
4,870486,2020-01,0
...,...,...,...
1537301,1200362290,2021-02,0
1537302,1200362291,2021-02,0
1537303,1200362301,2021-02,0
1537304,1200362302,2021-02,1


In [110]:
seg['PartyId'] = seg['PartyId'].astype('str')
ms_summary['PartyId'] = ms_summary['PartyId'].astype('str')
ms_summary = pd.merge(ms_summary, seg, how='inner',on=["PartyId", "month"])
ms_summary

,PartyId,month,월별리워드획득횟수,주간미션달성횟수,월별미션달성률,class
0,9141996,2020-11,4.0,4.0,100.0,0
1,46712094,2020-11,4.0,4.0,100.0,0
2,48541598,2020-11,4.0,4.0,100.0,0
3,49322580,2020-11,2.0,3.0,75.0,0
4,49322580,2020-11,0.0,3.0,75.0,0
...,...,...,...,...,...,...
98030,1200241790,2020-12,0.0,0.0,0.0,1
98031,1200251692,2020-12,0.0,0.0,0.0,0
98032,1200259349,2020-12,0.0,0.0,0.0,0
98033,1200279565,2020-12,0.0,1.0,100.0,0


In [33]:
ms_summary.to_csv('result/ms_summary.csv')

## 멤버데이터 
- 멤버
1. 앱푸쉬 여부(Y/N)
2. 월별 바이탈리티 측정횟수
3. 성별
4. 연령대
5. 현재 등급(4개), 1~4 부여
6. 누적월별 주간미션달성률